<a href="https://colab.research.google.com/github/fnmendez/buda.com_to_google_spreadsheet/blob/main/Buda_To_Spreadsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Buda.com a Google Spreadsheet con un click (read-only API-Key)

Este jupyter obtendrá todas las órdenes que has abierto en Buda en las que has pagado o recibido algún monto (parcial o completamente transadas), y las escribirá en el Google Spreadsheet que indiques.

A las órdenes les adjuntará los depósitos y retiros, adaptándolos a su formato. Por ejemplo, las órdenes tienen tipos "Bid" y "Ask", entonces añadimos "Dep" y "Wit" de Deposit y Withdrawal. De esta manera escribiremos todos los flujos de tus divisas de Buda en el spreadsheet.

La manera en la que se sobreescribe el spreadsheet te permitirá conservar los formatos que hayas definido, y no se eliminará el contenido que no se sobreescriba.

## 1. Autenticación

In [ ]:
# Reemplaza con los valores que correspondan

# El nombre de tu spreadsheet que debe estar en la misma carpeta que este archivo y ya creado
google_spreadsheet_name = "Your Buda Index"

# Tus claves de Buda.com: genéralas en Configuración > API Keys, no usaremos permisos de trading ni de retiros así que desactívalos
api_key = "your_api_key"
secret = "your_api_secret"

In [ ]:
# Ingresa sesión con tu cuenta Google (necesitarás hacer OAuth, mira los logs)

!pip install --upgrade gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

## 2. La magia

### 2.1 Este inciso solo es necesario ejecutarlo la primera vez

In [ ]:
import base64
import hmac
import time
import requests.auth
from datetime import datetime

In [ ]:
class BudaHMACAuth(requests.auth.AuthBase):
    """Adjunta la autenticación HMAC de Buda al objeto Request."""

    def __init__(self, api_key: str, secret: str):
        self.api_key = api_key
        self.secret = secret

    def get_nonce(self) -> str:
        # 1. Generar un nonce (timestamp en microsegundos)
        return str(int(time.time() * 1e6))

    def sign(self, r, nonce: str) -> str:
        # 2. Preparar string para firmar
        components = [r.method, r.path_url]
        if r.body:
            encoded_body = base64.b64encode(r.body).decode()
            components.append(encoded_body)
        components.append(nonce)
        msg = ' '.join(components)
        # 3. Obtener la firma
        h = hmac.new(key=self.secret.encode(),
                        msg=msg.encode(),
                        digestmod='sha384')
        signature = h.hexdigest()
        return signature

    def __call__(self, r):
        nonce = self.get_nonce()
        signature = self.sign(r, nonce)
        # 4. Adjuntar API-KEY, nonce y firma al header del request
        r.headers['X-SBTC-APIKEY'] = self.api_key
        r.headers['X-SBTC-NONCE'] = nonce
        r.headers['X-SBTC-SIGNATURE'] = signature
        return r

In [ ]:
# Utils para API de Buda

auth = BudaHMACAuth(api_key, secret)

# Obtener órdenes
def get_orders(market_id = 'btc-clp', state = None):
  url = f'https://www.buda.com/api/v2/markets/{market_id}/orders'
  response = requests.get(url, auth=auth, params={
      'state': state,
      'per': 300,
      'page': 1,
  })
  return response.json()["orders"]

# Obtener depósitos
def get_deposits(currency = 'btc', state = 'confirmed'):
  url = f'https://www.buda.com/api/v2/currencies/{currency}/deposits'
  response = requests.get(url, auth=auth, params={
      'state': state,
      'per': 300,
      'page': 1,
  })
  return response.json()["deposits"]

# Obtener retiros
def get_withdrawals(currency = 'btc', state = 'confirmed'):
  url = f'https://www.buda.com/api/v2/currencies/{currency}/withdrawals'
  response = requests.get(url, auth=auth, params={
      'state': state,
      'per': 300,
      'page': 1,
  })
  return response.json()["withdrawals"]

# Parsear depósitos o retiros como si fueran órdenes

def parseToOrder(kind, created_at, amount, currency, fee, state):
  ret = {}
  ret["created_at"] = created_at
  ret["type"] = kind
  ret["price_type"] = ""
  ret["state"] = state
  ret["original_amount"] = amount
  ret["traded_amount"] = amount
  ret["total_exchanged"] = ["", ""]
  ret["paid_fee"] = fee
  ret["amount"] = ["0.0", currency]
  ret["_price"] = ["", ""]
  return ret

In [ ]:
# Utils para API de Google Spreadsheets

# rowcol(1,1) => A1
rowcol = gspread.utils.rowcol_to_a1

# datetime parser
readtime = lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ") # TODO: aceptar distintos a Zulu time (+00:00)
writetime = lambda x: x.strftime("%Y-%m-%d %H:%M")

### 2.2 Ejecuta los siguientes bloques para actualizar el sheet

In [ ]:
# Obtenemos las órdenes y filtramos las que no se pagó/recibió algún monto

orders_raw = get_orders('btc-clp', None)
orders = list(filter(lambda x: float(x["total_exchanged"][0]) != 0.0, orders_raw))
for o in orders:
  o["_price"] = [
    float(o["total_exchanged"][0])/float(o["traded_amount"][0]),
    f'{o["total_exchanged"][1]}/{o["traded_amount"][1]}'
  ]

In [ ]:
# Obtenemos los depósitos y retiros y los parseamos para mezclarlos con las órdenes

deposits_raw = get_deposits('btc', 'confirmed')
deposits = []
for dr in deposits_raw:
  aux = parseToOrder("Dep", dr["created_at"], dr["amount"], dr["currency"], dr["fee"], dr["state"])
  deposits.append(aux)

withdrawals_raw = get_withdrawals('btc', 'confirmed')
withdrawals = []
for wr in withdrawals_raw:
  aux = parseToOrder("Wit", wr["created_at"], wr["amount"], wr["currency"], wr["fee"], wr["state"])
  withdrawals.append(aux)

In [ ]:
# Juntamos órdenes, depósitos y retiros en una sola variable

super_data = orders + deposits + withdrawals

In [ ]:
# Parámetros posicionales para el sheet
HRC = (1,1) # posición inicial de los headers
ORC = (HRC[0]+1,HRC[1]) # poosición inicial de las órdenes

# Abrimos el archivo de Google Spreadsheets, específicamente el primer sheet (la primera hoja) [puedes investigar cómo escribir en otra ;)]
worksheet = gc.open(google_spreadsheet_name).sheet1

In [ ]:
# Definir Headers

# (Texto-Celda, Nombre-Llave, Tipo)
HEADERS = [
    ("Fecha", "created_at", "date"),
    ("Tipo", "type", "string"),
    ("Modo", "price_type", "string"),
    ("Estado", "state", "string"),
    ("", "original_amount", "currency"),
    ("Monto Inicial", "original_amount", "amount"),
    ("", "traded_amount", "currency"),
    ("Monto Transado", "traded_amount", "amount"),
    ("", "total_exchanged", "currency"),
    ("Monto Pagado", "total_exchanged", "amount"),
    ("", "paid_fee", "currency"),
    ("Comisión", "paid_fee", "amount"),
    ("", "amount", "currency"),
    ("No Transado", "amount", "amount"),
    ("", "_price", "currency"),
    ("Precio", "_price", "amount") # este lo calculamos nosotros como exchanged/traded
] # unused: id, market_id, account_id, source, fee_currency, limit

# Tip: los `currency` y los `amount` son arreglos del estilo [5.54420000, "BTC"]

In [ ]:
# Escribimos los headers
header_cells = worksheet.range(f'{rowcol(HRC[0],HRC[1])}:{rowcol(HRC[0],HRC[1]+len(HEADERS)-1)}')
for i, header_cell in enumerate(header_cells):
  header_cell.value = HEADERS[i][0]
worksheet.update_cells(header_cells)

In [ ]:
# Escribimos los datos de las órdenes, depósitos y retiros
for i, sd in enumerate(super_data):
  sd_cells = worksheet.range(f'{rowcol(ORC[0]+i,ORC[1])}:{rowcol(ORC[0]+i,ORC[1]+len(HEADERS)-1)}')
  for j, sd_cell in enumerate(sd_cells):
    cur = HEADERS[j]
    if cur[2] == "date":
      sd_cell.value = writetime(readtime(super_data[i][cur[1]]))
    elif cur[2] == "string":
      sd_cell.value = super_data[i][cur[1]]
    elif cur[2] == "currency":
      sd_cell.value = super_data[i][cur[1]][1]
    elif cur[2] == "amount":
      sd_cell.value = super_data[i][cur[1]][0]
  # "USER_ENTERED" nos permite que el output considere el formato que estaba definido en la celda
  # si se desea que el output sea tal cual el input, usar "RAW"
  worksheet.update_cells(sd_cells, value_input_option="USER_ENTERED")

## 3. Trabajo futuro

- Crear spreadsheet si no existe
- Paginar resultados (si se tienen más de 300)
- Agregar análisis de los datos